In [1]:
## import 
import pandas as pd 
import numpy as np 
import requests
from bs4 import BeautifulSoup
import time 
import re
from random import randint
from warnings import warn
import json 

#### Scripts for scraping craigslist missed connections page 

In [2]:
url = "https://newyork.craigslist.org/search/mis#search=1~list~0~0"
r = requests.get(url) 
soup = BeautifulSoup(r.content, 'html.parser')
soup

<!DOCTYPE html>

<html>
<head>
<meta charset="utf-8"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<meta content="craigslist" property="og:site_name"/>
<meta content="preview" name="twitter:card"/>
<meta content="new york missed connections - craigslist" property="og:title"/>
<meta content="new york missed connections - craigslist" name="description"/>
<meta content="new york missed connections - craigslist" property="og:description"/>
<meta content="https://newyork.craigslist.org/search/mis" property="og:url"/>
<title>new york missed connections - craigslist</title>
<link href="https://newyork.craigslist.org/search/mis" rel="canonical"/>
<link href="https://newyork.craigslist.org/search/mis" hreflang="x-default" rel="alternate"/>
<link href="/favicon.ico" id="favicon" rel="icon">
<script id="ld_searchpage_data" type="application/ld+json">
    {"@type":"SearchResultsPage","@context":"https://schema.org","de

In [3]:
# get all the a tag (post links) in missed connection page
page = soup.find_all('a')
page

[<a href="#" id="cl-unrecoverable-hard-refresh" onclick="location.reload(true);">refresh the page.</a>,
 <a href="/">craigslist</a>,
 <a href="https://newyork.craigslist.org/mnh/mis/d/new-york-lower-east-sidearried-woan-for/7762703084.html">
 <div class="title">Lower East Side.......𝙼arried Wo𝙼an for 𝙼ale 💋</div>
 <div class="details">
 <div class="price">$0</div>
 <div class="location">
                         Lower East Side
                     </div>
 </div>
 </a>,
 <a href="https://newyork.craigslist.org/mnh/mis/d/new-york-looking-for-my-guy/7762696458.html">
 <div class="title">Looking for my guy</div>
 <div class="details">
 <div class="price">$0</div>
 <div class="location">
                         Upper East Side
                     </div>
 </div>
 </a>,
 <a href="https://newyork.craigslist.org/que/mis/d/howard-beach-my-fem-dom-friend-we-lost/7762671533.html">
 <div class="title">My fem-dom friend we lost contact</div>
 <div class="details">
 <div class="price">$0</div>
 <d

In [14]:
# get all the post links and save those in a dataframe
links = []
for i in page: 
    links.append(i.get('href'))
links
# # save all the links to a dataframe and to a csv? 
# df = pd.DataFrmae(links[2:]) 

['#',
 '/',
 'https://newyork.craigslist.org/mnh/mis/d/new-york-lower-east-sidearried-woan-for/7762703084.html',
 'https://newyork.craigslist.org/mnh/mis/d/new-york-looking-for-my-guy/7762696458.html',
 'https://newyork.craigslist.org/que/mis/d/howard-beach-my-fem-dom-friend-we-lost/7762671533.html',
 'https://newyork.craigslist.org/mnh/mis/d/new-york-tonights-bedroom-enjoyment/7762667561.html',
 'https://newyork.craigslist.org/mnh/mis/d/new-york-ready-for-hotel-adventure/7762667483.html',
 'https://newyork.craigslist.org/mnh/mis/d/new-york-running-hudson-river-greenway/7762649409.html',
 'https://newyork.craigslist.org/brx/mis/d/bronx-yankee-stadium-area-crossdressr/7762600368.html',
 'https://newyork.craigslist.org/mnh/mis/d/new-york-looking-for-cute-asian-woman-sw/7762565646.html',
 'https://newyork.craigslist.org/mnh/mis/d/new-york-relax-fully-in-private-and/7762589442.html',
 'https://newyork.craigslist.org/que/mis/d/jackson-heights-you-were-standing-firm/7762571970.html',
 'https

In [6]:
# test to get one post detail
onePage = requests.get(links[3])

print(f"--------getting request from {onePage}")
onePage_soup = BeautifulSoup(onePage.content)
print(f"--------getting title")
title = onePage_soup.find('span', class_='postingtitletext').text #'str' object is not callable
print('title:' + title)
# get map lat and lng 
print(f"--------getting lat and lng")
geos = onePage_soup.findAll("div", {"class": "mapbox"})
lat = geos[0].contents[1].get('data-latitude')
print('lat:' + lat.strip())
long = geos[0].contents[1].get('data-longitude')
print('long:' + long.strip())

# get description
print(f"--------getting description")
post_body = onePage_soup.find(attrs={'id' : 'postingbody'}).contents[2]
print('post_body:' + re.sub("[^0-9a-zA-Z]+", " ", post_body))

--------getting request from <Response [200]>
--------getting title
title:
Looking for my guy (Upper East Side) 
--------getting lat and lng
lat:40.768500
long:-73.958800
--------getting description
post_body: What was your nickname for me It had something to do with chocolate bar I will know this if it is you have a new number reach out


In [20]:
## remove the first two link in links because they were "#" for some reason
## get all the post details and store it in mconnections []

mconnections = []

for link in links[2:]: 
    # make http request
    each_page = requests.get(link)
    # sleep function to avoid overloading the server
    time.sleep(randint(1,2))
    # store the BS object in a variable 
    page_soup = BeautifulSoup(each_page.content, 'html.parser')
    # loop over each link to missed connections details 
    print('-----')
    print(f"getting this {link}")
    try: 
        print(f"--------getting title")
        title = page_soup.find('span', class_='postingtitletext').text
        print('title:' + title)
        
        # get map lat and lng 
        print(f"--------getting lat and lng")
        geos = page_soup.findAll("div", {"class": "mapbox"})
        lat = geos[0].contents[1].get('data-latitude')
        print('lat:' + lat.strip())
        long = geos[0].contents[1].get('data-longitude')
        print('long:' + long.strip())
        
        # get description
        print(f"--------getting description")
        post_body = page_soup.find(attrs={'id' : 'postingbody'}).contents[2]
        # print('post_body:' + re.sub("[^0-9a-zA-Z]+", " ", post_body))
        post_clean = re.sub("[^0-9a-zA-Z]+", " ", post_body)
        print('clean_post' + post_clean)

        date = page_soup.find('time', class_='date timeago').text.strip()
        print('date:' + date)
        
        mconnections.append({
            'url': link,
            'date': date,
            'title': title,
            'lat': lat,
            'lng': long, 
            'post': post_clean,
        })
        
    except: 
        print('******story not found*******')
        pass

print(mconnections)

-----
getting this https://newyork.craigslist.org/mnh/mis/d/new-york-lower-east-sidearried-woan-for/7762703084.html
--------getting title
title:
Lower East Side.......𝙼arried Wo𝙼an for 𝙼ale 💋 (Lower East Side) 
--------getting lat and lng
lat:40.824600
long:-73.950700
--------getting description
clean_post Text call 
date:2024-07-03 09:30
-----
getting this https://newyork.craigslist.org/mnh/mis/d/new-york-looking-for-my-guy/7762696458.html
--------getting title
title:
Looking for my guy (Upper East Side) 
--------getting lat and lng
lat:40.768500
long:-73.958800
--------getting description
clean_post What was your nickname for me It had something to do with chocolate bar I will know this if it is you have a new number reach out
date:2024-07-03 09:04
-----
getting this https://newyork.craigslist.org/que/mis/d/howard-beach-my-fem-dom-friend-we-lost/7762671533.html
--------getting title
title:
My fem-dom friend we lost contact 
--------getting lat and lng
lat:40.653177
long:-73.828125
--

In [21]:
df_mconnections = pd.DataFrame(mconnections)
print(df_mconnections)
df_mconnections.to_csv('mconnections_july_3_wtime.csv', sep='\t', encoding='utf-8')

                                                   url              date  \
0    https://newyork.craigslist.org/mnh/mis/d/new-y...  2024-07-03 09:30   
1    https://newyork.craigslist.org/mnh/mis/d/new-y...  2024-07-03 09:04   
2    https://newyork.craigslist.org/que/mis/d/howar...  2024-07-03 05:46   
3    https://newyork.craigslist.org/mnh/mis/d/new-y...  2024-07-03 04:26   
4    https://newyork.craigslist.org/mnh/mis/d/new-y...  2024-07-03 04:25   
..                                                 ...               ...   
232  https://newyork.craigslist.org/stn/mis/d/state...  2024-06-03 09:32   
233  https://newyork.craigslist.org/mnh/mis/d/new-y...  2024-06-03 07:33   
234  https://newyork.craigslist.org/mnh/mis/d/new-y...  2024-06-02 22:44   
235  https://newyork.craigslist.org/mnh/mis/d/new-y...  2024-06-02 22:42   
236  https://newyork.craigslist.org/que/mis/d/astor...  2024-06-02 22:40   

                                                 title        lat         lng  \
0    \

#### Scraping overheardnewyork's instagram stories to find out if missed connections were ever resolved with instaloader(commandline)

In [ ]:
!instaloader --login=['username'] --password=['password'] --profile overheardnewyork --stories

#### Scraping internet archive - borrow jeremy's waybackpack (https://github.com/jsvine/waybackpack)

In [ ]:
# !pip install waybackpack

In [ ]:
# list of url strings for new york missed connection page on wayback
wayback = !waybackpack https://newyork.craigslist.org/mis/ --list
# write url to a dataframe 
url_pd = pd.DataFrame(wayback)
# url_pd.to_csv('waybackurl.csv', sep=',', index=False)

In [ ]:
# find out how to srape the first url from internet archive --- where i scream a lot...
url = wayback[0]
print(f'url:\n{url}')
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')
print(soup.prettify())
# use beautifulSoup to scrape first page

block = soup.find('blockquote')
block

## wrote some really ugly parsing code to attempt to achive the following
## ideal data format: 
# [
#     {
#         'date': 'Thu Feb 15th', 
#         'title': 'cnr 42 & 9th, by Citibank at approx. 7:30pm 02-16-01 (new york)'
#     }, 
#     {
#         'date': 'Thu Feb 15th', 
#         'title': 'tony perrota '
#     }
# ]

In [ ]:
## jeremy came to rescue:
## still need to figure out connection timeout error and terrified that I cant debug anything so i did it manually...
# for i in range(0, 609, 15): 
#     print(i, i+15)
## scraping all the internet archive url to get date and post 

for i in range(len(df)):
    print(f'{i} ----- getting url {df.iloc[i, 0]}')
    url = df.iloc[i, 0]
    r = requests.get(url)
    time.sleep(randint(1,10))
    soup = BeautifulSoup(r.content)
    block = soup.select('blockquote')

    modified_date = 'modified_date'
    # print(f"modified_date: \n{modified_date}")

    current_date = ""
    for el in soup.select("blockquote td, blockquote p a"):
        if el.name == "td":
            current_date = el.text.strip()
        else:
            post_text = el.text.strip()
            dict = {
                "modified_date": modified_date, 
                "url": url,
                "date": current_date,
                "text": post_text
            }
            data.append(dict)


data_csv = pd.DataFrame(data)
data_csv.to_csv('wayback_test_csv', sep=',', index=False)
print('csv in the folder')